<a href="https://colab.research.google.com/github/FrancescoPassiatore/2017-09-22/blob/master/gesture_recognition_3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Scaricare e aprire acquisizioni.zip
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/MyDrive/acquisizioni.zip" -d "/content/dataset"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%bash
pip install mediapipe

In [ ]:
import os
import numpy as np
import cv2
import mediapipe as mp
import csv

# MediaPipe setup
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=True,
    max_num_hands=1,
)

# Percorsi
DATASET_DIR = "/content/3"  # Cambia se necessario
output_npy = "landmarks.npy"
output_labels = "labels.npy"
output_csv = "landmarks.csv"

# Output
all_landmarks = []
all_labels = []

# Mapping label
label_map = {
    "1": 1,
    "2": 2,
    "tre": 3,
    "0": 0
}

# CSV setup
csv_file = open(output_csv, mode='w', newline='')
csv_writer = csv.writer(csv_file)

# Header
header = [f'{dim}{i}' for i in range(21) for dim in ['x', 'y', 'z']]
header.append('label')
csv_writer.writerow(header)

# Contatori
total_images = 0
processed_images = 0
skipped_images = 0

# Scansiona le sottocartelle
for class_name in sorted(os.listdir(DATASET_DIR)):
    print(f"✔️ Cartella: {class_name}")
    class_path = os.path.join(DATASET_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    if class_name not in label_map:
        print(f"⚠️ La cartella '{class_name}' non ha una label definita. Saltata.")
        continue
    label = label_map[class_name]

    for filename in os.listdir(class_path):
        print(filename)
        if not filename.endswith('_Color.png'):
            continue  # ⛔ Salta file che non terminano in _Color.png

        total_images += 1

        image_path = os.path.join(class_path, filename)
        image = cv2.imread(image_path)
        if image is None:
            print(f"❌ Errore nel caricamento di: {image_path}")
            skipped_images += 1
            continue

        # Converti BGR in RGB per MediaPipe
        image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        results = hands.process(image_rgb)

        if results.multi_hand_landmarks:
            hand = results.multi_hand_landmarks[0]
            landmarks = []
            for lm in hand.landmark:
                landmarks.extend([lm.x, lm.y, lm.z])
            all_landmarks.append(landmarks)
            all_labels.append(label)
            csv_writer.writerow(landmarks + [label])
            processed_images += 1
        else:
            print(f"🙈 Nessuna mano rilevata in: {image_path}")
            skipped_images += 1

# Cleanup
csv_file.close()
hands.close()

np.save(output_npy, np.array(all_landmarks))
np.save(output_labels, np.array(all_labels))

# Report finale
print(f"\n✅ Salvati: {output_npy}, {output_labels}, {output_csv}")
print(f"📊 Totale immagini trovate: {total_images}")
print(f"👌 Immagini elaborate con successo: {processed_images}")
print(f"🚫 Immagini scartate (errore o mano non rilevata): {skipped_images}")
